### ____init____ialisieren der benötigten Klassen
Im nächsten Schritt werden die Klassen definiert, deren Objekte in der CSV beschrieben werden.

In [6]:
class Task:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Activity:
    def __init__(self, ID, name, parentTask, semaphoresIN, semaphoresOUT, mutexes):
        self.ID = ID
        self.name = name
        self.parentTask = parentTask
        self.semaphoresIN = semaphoresIN
        self.semaphoresOUT = semaphoresOUT
        self.mutexes = mutexes

class Mutex:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Semaphore:
    def __init__(self, ID, name, initialValue, activityIN, activityOUT):
        self.ID = ID
        self.name = name
        self.initialValue = initialValue
        self.activityIN = activityIN
        self.activityOUT = activityOUT

### Test der Klassen
Um die Klassen zu testen wird das folgende Diagramm aufgebaut: 

![Fehler beim Abrufen der Datei: ./img/test_diagram.png](./img/test_diagram.png)

In [7]:
task1 = Task(1, "Task1", [1])
task2 = Task(2, "Task2", [2])
task3 = Task(3, "Task3", [3])
task4 = Task(4, "Task4", [4])
task5 = Task(5, "Task5", [5, 7])
task6 = Task(6, "Task6", [6])

activity1 = Activity(1, "Activity1", 1, [12, 13], [])
activity2 = Activity(2, "Activity2", 2, [24], [234])
activity3 = Activity(3, "Activity3", 3, [36], [234])
activity4 = Activity(4, "Activity4", 4, [46], [234])
activity5a = Activity(5, "Activity5a", 5, [55], [])
activity5b = Activity(7, "Activity5b", 5, [56], [])
activity6 = Activity(6, "Activity6", 6, [65], [])

mutex234 = Mutex(234, "Mutex234", [2, 3, 4])

semaphore12 = Semaphore(12, "Semaphore12", [2], None)
semaphore13 = Semaphore(13, "Semaphore13", [3], None)
semaphore24 = Semaphore(24, "Semaphore24", [4], None)
semaphore36 = Semaphore(36, "Semaphore36", [6], None)
semaphore46 = Semaphore(46, "Semaphore46", [6], None)
semaphore55 = Semaphore(55, "Semaphore55", [7], 1)
semaphore56 = Semaphore(56, "Semaphore56", [5], None)
semaphore65 = Semaphore(65, "Semaphore65", [5], 1)

TypeError: Activity.__init__() missing 1 required positional argument: 'mutexes'

### Einlesen der CSV Datei
Als nächstes sollen die Objekte aus einer CSV Datei eingelesen werden.

In [8]:
dataFilePath = "./dataTest.csv"

data = []
linecells = []

import pandas as pd
df = pd.read_csv(dataFilePath, sep=',')

In [9]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,None,12;13,Semaphore12;Semaphore13,None;None,51,None,None
1,2,Task2,2,Activity2,None,24,Semaphore24,None,12,234,Mutex234
2,3,Task3,3,Activity3,None,36,Semaphore36,None,13,234,Mutex234
3,4,Task4,4,Activity4,None,46,Semaphore46,None,24,234,Mutex234
4,5,Task5,5,Activity5a,None,55,Semaphore55,None,[56;65],None,None
5,5,Task5,7,Activity5b,None,51;56,Semaphore51;Semaphore56,1;1,55,None,None
6,6,Task6,6,Activity6,None,65,Semaphore65,None,36;46,None,None


In [10]:
# split seperated values into list elements
if not isinstance(df["Semaphore_ID"][0], list):
    df["Semaphore_ID"] = df["Semaphore_ID"].str.split(";")
if not isinstance(df["Semaphore_Name"][0], list):
    df["Semaphore_Name"] = df["Semaphore_Name"].str.split(";")
if not isinstance(df["Semaphore_Initial_Value"][0], list):
    df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].str.split(";")
if not isinstance(df["Predecessor_Semaphore_ID"][0], list):
    df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].str.split(";")
if not isinstance(df["Mutex_ID"][0], list):
    df["Mutex_ID"] = df["Mutex_ID"].str.split(";")
if not isinstance(df["Mutex_Name"][0], list):
    df["Mutex_Name"] = df["Mutex_Name"].str.split(";")

In [11]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,None,"[12, 13]","[Semaphore12, Semaphore13]","[None, None]",[51],[None],[None]
1,2,Task2,2,Activity2,None,[24],[Semaphore24],[None],[12],[234],[Mutex234]
2,3,Task3,3,Activity3,None,[36],[Semaphore36],[None],[13],[234],[Mutex234]
3,4,Task4,4,Activity4,None,[46],[Semaphore46],[None],[24],[234],[Mutex234]
4,5,Task5,5,Activity5a,None,[55],[Semaphore55],[None],"[[56, 65]]",[None],[None]
5,5,Task5,7,Activity5b,None,"[51, 56]","[Semaphore51, Semaphore56]","[1, 1]",[55],[None],[None]
6,6,Task6,6,Activity6,None,[65],[Semaphore65],[None],"[36, 46]",[None],[None]


In [12]:
# only during the creation process of the tree
tasks_IDs = []
tasks = []

activities_IDs = []
activities = []

semaphore_IDs = []
semaphores = []

mutex_IDs = []
mutexs = []

for index, row in df.iterrows():
    # first: instantiation of objects
    if row["Task_ID"] not in tasks_IDs:
        tasks_IDs.append(row["Task_ID"])
        tasks.append(Task(row["Task_ID"], row["Task_Name"], []))
    
    if row["Activity_ID"] not in activities_IDs:
        activities_IDs.append(row["Activity_ID"])
        activities.append(Activity(row["Activity_ID"], row["Activity_Name"], [], row["Predecessor_Semaphore_ID"], [], []))
    
    for index, semaphore_ID in enumerate(row["Semaphore_ID"]):
        if not semaphore_ID == "None" and semaphore_ID not in semaphore_IDs:
            semaphore_IDs.append(semaphore_ID)
            semaphores.append(Semaphore(semaphore_ID, row["Semaphore_Name"][index], row["Semaphore_Initial_Value"][index], [], []))
        
    for index, mutex_ID in enumerate(row["Mutex_ID"]):
        if not mutex_ID == "None" and mutex_ID not in mutex_IDs:
            mutex_IDs.append(mutex_ID)
            mutexs.append(Mutex(mutex_ID, row["Mutex_Name"][index], []))
            
    # add semaphoreIN to activities and semaphores
    
        
    # secondly: linking of objects
    tasks[tasks_IDs.index(row["Task_ID"])].activities.append(activities[activities_IDs.index(row["Activity_ID"])])
    
    activities[activities_IDs.index(row["Activity_ID"])].parentTask = tasks[tasks_IDs.index(row["Task_ID"])]
    for semaphore_ID in row["Semaphore_ID"]:
        if not semaphore_ID == "None":
            activities[activities_IDs.index(row["Activity_ID"])].semaphoresOUT.append(semaphores[semaphore_IDs.index(semaphore_ID)])
            semaphores[semaphore_IDs.index(semaphore_ID)].activityOUT = activities[activities_IDs.index(row["Activity_ID"])]
    for mutex_ID in row["Mutex_ID"]:
        if not mutex_ID == "None":
            activities[activities_IDs.index(row["Activity_ID"])].mutexes.append(mutexs[mutex_IDs.index(mutex_ID)])
            mutexs[mutex_IDs.index(mutex_ID)].activities.append(activities[activities_IDs.index(row["Activity_ID"])])
            
# add semaphoreIN to activities and semaphores
for activity in activities:
    buf = activity.semaphoresIN
    activity.semaphoresIN = []
    for semaphore_id in buf:
        activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
        semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity

ValueError: '[56' is not in list

In [13]:
root = tasks[0]
print("hallo")

hallo


In [16]:
print("Task 1 ist mit folgendem Task verbunden: Task " + str(tasks[2].activities[0].semaphoresOUT[0].activityIN.parentTask.ID))

AttributeError: 'list' object has no attribute 'parentTask'